In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import seaborn as sns
import os

from sklearn.model_selection import KFold, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMRegressor
from tqdm import tqdm
import optuna
from functools import partial
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
t1=[[1,1,1],[3,3,3]]
t2=[[1,1,1],[3,3,3]]
test_t=[]
test_t[len(test_t):] = t1
test_t[len(test_t):] = t2
np.save(f'test.npy', test_t)
test_l = np.load('test.npy')
np.array(test_l).mean(axis=0)

array([2., 2., 2.])

In [3]:
train = pd.read_csv('..\\kaggle_data\\train.csv')
test = pd.read_csv('..\\kaggle_data\\test.csv')

In [4]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train.target
X_test = test.drop(['id'], axis=1)

In [5]:
cat_cols = [feature for feature in train.columns if 'cat' in feature]

def label_encoder(df):
    for feature in cat_cols:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [6]:
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

In [7]:
split = KFold(n_splits=5)

In [40]:
preds_list_final_iterations = []
preds_list_final_iteration = []
study_lgbm = optuna.create_study(direction='maximize')

root_directory = 'N:\\kaggle_data\\march'
wave = '7'

preds_target = f'{root_directory}\\w{wave}_preds'
subs_target = f'{root_directory}\\w{wave}_subs'

if not os.path.exists(preds_target):
    os.makedirs(preds_target)
    
if not os.path.exists(subs_target):
    os.makedirs(subs_target)

[I 2021-03-14 16:33:14,466] A new study created in memory with name: no-name-fe25e5cc-d451-4900-97ee-a20ab7c0a1ec


In [41]:
def objective(trial, X, y, name='xgb'):
            
    if trial.number % 5 == 0:
        clear_output(wait=True)
    
    params = {'max_depth':trial.suggest_int('max_depth', 5, 50),
              'n_estimators':200000,
              #'boosting':trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
              'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
              'learning_rate':trial.suggest_uniform('learning_rate', 0.007, 0.02),
              'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 50),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.01, 50),
              'min_child_samples':trial.suggest_int('min_child_samples', 5, 100),
              'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              'n_jobs' : -1,
              'metric':'AUC',
              'max_bin':trial.suggest_int('max_bin', 300, 1000),
              'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)}

    model = LGBMRegressor(**params)
                  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    
    print(f'Fit Starting')
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
              eval_metric=['AUC'],
              early_stopping_rounds=250, 
              categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
              #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
              verbose=0)
    
    print(f'Appending to List')
    preds_list_final_iterations.append(model.predict(X_test))

    dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    #np.save(f'..\\kaggle_data\\{dt_string}_FI{fold_index}_sub.npy', preds_list_final_iterations)
    print(f'Calculating Mean')
    y_preds_final_iteration = np.array(preds_list_final_iterations).mean(axis=0)
    print(f'length of infal list:{len(preds_list_final_iterations)}')
    submission = pd.DataFrame({'id':test.id,
              'target':y_preds_final_iteration})
    file_target = f'{root_directory}\\w{wave}_subs\\{dt_string}_AUTOML.csv'
    submission.to_csv(file_target, index=False)
    print(f'written to CSV')
    
    np.save(f'{root_directory}\\w{wave}_preds\\_preds_list_final_iterations.npy', preds_list_final_iterations)
    np.save(f'{root_directory}\\w{wave}_preds\\_preds_list_final_iteration_scores.npy', preds_list_final_iteration_scores)
    
    print(f'written to npy')

    #train_score = np.round(np.sqrt(mean_squared_error(y_train, model.predict(X_train))), 15)
    test_score = roc_auc_score(y_val, model.predict(X_val))
    
    preds_list_final_iteration_scores.append(test_score)
    print(f'TEST AOC : {test_score}')
                  
    return test_score

In [ ]:
optimize = partial(objective, X=X_train, y=y_train)
study_lgbm.optimize(optimize, n_trials=1000)

best_trial = study_lgbm.best_trial.params

optuna.visualization.plot_optimization_history(study_lgbm)

clear_output(wait=True)

print(f'best trial: {best_trial}')
# i have commented out the trials so as to cut short the notebook execution time.

Fit Starting
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Appending to List
Calculating Mean
length of infal list:21
written to CSV
written to npy


[I 2021-03-14 16:49:19,073] Trial 20 finished with value: 0.8916010725276675 and parameters: {'max_depth': 5, 'subsample': 0.7354857143139755, 'colsample_bytree': 0.9293965423574082, 'learning_rate': 0.017100822810724538, 'reg_lambda': 24.991662178573044, 'reg_alpha': 29.25012965581383, 'min_child_samples': 8, 'num_leaves': 31, 'max_bin': 514, 'cat_smooth': 42, 'cat_l2': 5.097628245558588}. Best is trial 11 with value: 0.898169453130433.


TEST RMSE : 0.8916010725276675
Fit Starting
Appending to List
Calculating Mean
length of infal list:22
written to CSV
written to npy


[I 2021-03-14 16:50:01,644] Trial 21 finished with value: 0.8942313730989255 and parameters: {'max_depth': 22, 'subsample': 0.9317904265116425, 'colsample_bytree': 0.2882462423464595, 'learning_rate': 0.010371158064875353, 'reg_lambda': 34.980043843345854, 'reg_alpha': 5.607623050293053, 'min_child_samples': 41, 'num_leaves': 158, 'max_bin': 995, 'cat_smooth': 99, 'cat_l2': 1.9794940836792627}. Best is trial 11 with value: 0.898169453130433.


TEST RMSE : 0.8942313730989255
Fit Starting


In [65]:
root_directory = 'N:\\kaggle_data\\march'

fold_ittr_range = 20
preds_list_final_iterations =[]

for fold_index in range(2, fold_ittr_range):
    print(f'Current fold index {fold_index}')
    split = KFold(n_splits=fold_index)
    lgbm_params = {'max_depth': 33, 'subsample': 0.2028128133863628, 'colsample_bytree': 0.3316256936017043, 'learning_rate': 0.007295415117417998, 'reg_lambda': 21.211226081349032, 'reg_alpha': 7.386690008287902, 'min_child_samples': 88, 'num_leaves': 200, 'max_bin': 859, 'cat_smooth': 100, 'cat_l2': 3.5709736612307674,
                'verbose':-1}
    

    preds_list_base = []
    preds_list_final_iteration = []
    preds_list_all = []

    for train_idx, val_idx in tqdm(split.split(X_train)):
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]

        Model = LGBMRegressor(**lgbm_params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                      eval_metric=['rmse'],
                      early_stopping_rounds=250, 
                      categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                      #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                      verbose=0)

        preds_list_base.append(Model.predict(X_test))
        preds_list_all.append(Model.predict(X_test))
        print(f'RMSE for Base model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
        first_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
        params = lgbm_params.copy()

        for i in range(1, 8):
            if i >2:    

                # reducing regularizing params if 

                params['reg_lambda'] *= 0.9
                params['reg_alpha'] *= 0.9
                params['num_leaves'] += 40

            params['learning_rate'] = 0.003
            Model = LGBMRegressor(**params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                      eval_metric=['rmse'],
                      early_stopping_rounds=200, 
                      categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                      #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
                      verbose=0,
                      init_model=Model)

            preds_list_all.append(Model.predict(X_test))
            print(f'RMSE for Incremental trial {i} model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
        last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
        print('',end='\n\n')
        print(f'Improvement of : {first_rmse - last_rmse}')
        print('-' * 100)
        preds_list_final_iteration.append(Model.predict(X_test))
    preds_list_final_iterations[len(preds_list_final_iterations):] = preds_list_final_iteration
    dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    y_preds_final_iteration = np.array(preds_list_final_iterations).mean(axis=0)
    submission = pd.DataFrame({'id':test.id,
              'target':y_preds_final_iteration})
    submission.to_csv(f'{root_directory}\\wave6_{dt_string}_FI{fold_index}_sub.csv', index=False)

0it [00:00, ?it/s]

Current fold index 2
RMSE for Base model is 0.3738234021167528
RMSE for Incremental trial 1 model is 0.36010860301330344
RMSE for Incremental trial 2 model is 0.35097281486609255
RMSE for Incremental trial 3 model is 0.34481002483978457
RMSE for Incremental trial 4 model is 0.3406066612103717
RMSE for Incremental trial 5 model is 0.33769926441221504
RMSE for Incremental trial 6 model is 0.3356322816657475
RMSE for Incremental trial 7 model is 0.33413741053308854


Improvement of : 0.03968599158366426
----------------------------------------------------------------------------------------------------


1it [00:38, 38.64s/it]

RMSE for Base model is 0.3732552875649722
RMSE for Incremental trial 1 model is 0.35974007951171033
RMSE for Incremental trial 2 model is 0.3507401606988118
RMSE for Incremental trial 3 model is 0.34467393254458034
RMSE for Incremental trial 4 model is 0.3405483626946245
RMSE for Incremental trial 5 model is 0.33769489293493155
RMSE for Incremental trial 6 model is 0.33568076390319496
RMSE for Incremental trial 7 model is 0.334219950642155


Improvement of : 0.03903533692281724
----------------------------------------------------------------------------------------------------


2it [01:16, 38.35s/it]
0it [00:00, ?it/s]

...

17it [10:13, 36.02s/it]

RMSE for Base model is 0.3730311956268487
RMSE for Incremental trial 1 model is 0.359844723288138
RMSE for Incremental trial 2 model is 0.3511707171323228
RMSE for Incremental trial 3 model is 0.34537932749875727
RMSE for Incremental trial 4 model is 0.3414717184131639
RMSE for Incremental trial 5 model is 0.3387694966743379
RMSE for Incremental trial 6 model is 0.33687092866174595
RMSE for Incremental trial 7 model is 0.335511068375593


Improvement of : 0.0375201272512557
----------------------------------------------------------------------------------------------------


18it [10:49, 35.97s/it]

RMSE for Base model is 0.3714718134467412
RMSE for Incremental trial 1 model is 0.357562080908172
RMSE for Incremental trial 2 model is 0.34830196116155315
RMSE for Incremental trial 3 model is 0.34205436305758397
RMSE for Incremental trial 4 model is 0.3378010110872906
RMSE for Incremental trial 5 model is 0.33486029907832787
RMSE for Incremental trial 6 model is 0.3327849962296434
RMSE for Incremental trial 7 model is 0.3312922922824477


Improvement of : 0.040179521164293464
----------------------------------------------------------------------------------------------------


19it [11:25, 36.07s/it]
